In [7]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris_data=load_iris()

In [8]:
# Features names and target values
print("Feature names:", iris_data.feature_names)
print("First 5 rows of features:\n", iris_data.data[:5])
print("\nTarget names (species):", iris_data.target_names)
print("First 5 target values:", iris_data.target[:5])

Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
First 5 rows of features:
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]

Target names (species): ['setosa' 'versicolor' 'virginica']
First 5 target values: [0 0 0 0 0]


In [9]:
# We split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=42)

In [10]:
# We can test a prediction model
# We are going to use SVM:

from sklearn import svm
from sklearn.metrics import accuracy_score

model=svm.SVC(probability=True)
model.fit(X_train,y_train)
y_pred=model.predict_proba(X_test)
print(f"Probability of each class:\n {y_pred}")
y_pred=model.predict(X_test)
accuracy=accuracy_score(y_pred,y_test)
print(f"Accuracy: {accuracy}")

Probability of each class:
 [[0.00853766 0.87223876 0.11922357]
 [0.93745199 0.04275456 0.01979345]
 [0.01838676 0.00313397 0.97847927]
 [0.009174   0.87123835 0.11958765]
 [0.01177556 0.7521716  0.23605284]
 [0.94450895 0.03912324 0.01636781]
 [0.028569   0.96213989 0.00929111]
 [0.01301524 0.07560238 0.91138237]
 [0.00795508 0.82087085 0.17117407]
 [0.01420277 0.97647892 0.00931831]
 [0.01216931 0.14658708 0.84124361]
 [0.95745788 0.02882119 0.01372092]
 [0.96373401 0.02263448 0.01363151]
 [0.95319954 0.03251004 0.01429042]
 [0.96329943 0.02263456 0.01406601]
 [0.01331256 0.75153395 0.2351535 ]
 [0.00819663 0.00589657 0.9859068 ]
 [0.01291605 0.9776028  0.00948115]
 [0.00858012 0.90658325 0.08483663]
 [0.00823354 0.01008332 0.98168314]
 [0.94829191 0.03641293 0.01529515]
 [0.01203078 0.36249716 0.62547206]
 [0.94435896 0.0395239  0.01611715]
 [0.0083363  0.01296982 0.97869388]
 [0.02076522 0.01102071 0.96821407]
 [0.01093391 0.04501506 0.94405103]
 [0.00903862 0.01373408 0.97722729]


The model seems to be extremely efficient to predict the type of petal since the probabilities are really distributed

We are going to save our model in a joblib file

In [11]:
from joblib import dump,load

dump(model,'iris_model_svm.joblib')

['iris_model_svm.joblib']

Now lets try a logistic regression

In [12]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=200)

# Train the model on the training set
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 100.00%


Save this model in a joblib file

In [13]:
from joblib import dump,load

dump(model,'iris_model_regression.joblib')

['iris_model_regression.joblib']

Random Forest Classifier

In [14]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred_probas = rf_classifier.predict_proba(X_test)
print('Predictions:', y_pred_probas)

y_pred = rf_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Predictions: [[0.   0.97 0.03]
 [1.   0.   0.  ]
 [0.   0.02 0.98]
 [0.   0.99 0.01]
 [0.   0.92 0.08]
 [1.   0.   0.  ]
 [0.   1.   0.  ]
 [0.   0.07 0.93]
 [0.   0.85 0.15]
 [0.   1.   0.  ]
 [0.   0.09 0.91]
 [1.   0.   0.  ]
 [0.95 0.05 0.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [0.   0.87 0.13]
 [0.   0.   1.  ]
 [0.   1.   0.  ]
 [0.   0.98 0.02]
 [0.   0.   1.  ]
 [1.   0.   0.  ]
 [0.   0.07 0.93]
 [1.   0.   0.  ]
 [0.   0.   1.  ]
 [0.   0.02 0.98]
 [0.   0.02 0.98]
 [0.   0.03 0.97]
 [0.   0.   1.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [0.01 0.99 0.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [0.   0.06 0.94]
 [0.   0.98 0.02]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [0.   0.05 0.95]
 [0.02 0.9  0.08]
 [0.   1.   0.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]]
Model accuracy: 100.00%


Save the model

In [15]:
from joblib import dump,load

dump(model,'iris_model_random_forest.joblib')

['iris_model_random_forest.joblib']

Simple API to make requests on with the 3 models combined

In [34]:
from flask import Flask, request, jsonify, render_template
from joblib import load
import numpy as np
from sklearn.datasets import load_iris

app = Flask(__name__)

# Load the model
model_svm = load('iris_model_svm.joblib')
model_regression = load('iris_model_regression.joblib')
model_randomF = load('iris_model_random_forest.joblib')

# Make sure to load the iris dataset for target_names (assuming it's not already loaded)
iris_data = load_iris()

@app.route("/")
def read_root():
    return render_template('base.html')


@app.route('/predict/', methods=['GET'])
def predict():
    try:
        L1 = float(request.args.get('L1'))
        W1 = float(request.args.get('W1'))
        L2 = float(request.args.get('L2'))
        W2 = float(request.args.get('W2'))

        testData = np.array([L1, W1, L2, W2]).reshape(-1, 4)

        predict1=model_svm.predict_proba(testData)[0]
        predict2=model_randomF.predict_proba(testData)[0]
        predict3=model_regression.predict_proba(testData)[0]
        
        # We have the predictions of our 3 models now let's mix them to return a mix
        setosa=(predict1[0]+predict2[0]+predict3[0])/3
        versicolor=(predict1[1]+predict2[1]+predict3[1])/3
        virginica=(predict1[2]+predict2[2]+predict3[2])/3

        results={'setosa':setosa,'versicolor':versicolor,'virginica':virginica}
        class_predicted=max(results, key=results.get)

        # Standardized API response format
        response = {
            "prediction": class_predicted,
            "probabilities": results
        }

    except Exception as e:
        response = {
            "message": str(e)
        }

    return jsonify(response)
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [19/Feb/2024 22:14:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2024 22:14:14] "GET /predict/?L1=2.5&W1=5&L2=4.2&W2=1.5 HTTP/1.1" 200 -
